### Helper functions:

In [1]:
def clear_issuer_name(name):
    name = name.replace(' CORPORATION', '')
    name = name.replace(' CORP', '')
    name = name.replace(' GROUP', '')
    name = name.replace(' INC', '')
    name = name.replace(' CO', '')
    name = name.replace(' S.A.', '')
    name = name.replace(' SA', '')
    name = name.replace(' LP', '')
    name = name.replace(' LTD', '')
    name = name.replace(' LLC', '')
    name = name.replace(' /DE/', '')
    name = name.replace(' /CAN/', '')
    name = name.replace(' LP', '')
    name = name.replace(' /MD/', '')
    name = name.replace(', LP', '')
    name = name.replace(', LP', '')
    name = name.replace(' PLC', '')
    name = name.replace(' LP', '')


    if name.endswith('.'):
        name = name.replace('.', '')
    
    if name.endswith(','):
        name = name.replace(',', '')
    
    return name

def get_feats(d, feats):
    return {feat: getattr(d, feat) for feat in feats}

### Loader function:

In [4]:
from collections import defaultdict
import sys
from itertools import groupby
import numpy as np
import pandas as pd
import os
import gc
from io import StringIO
import datetime

# def load_dataset(fin_data_path, rating_path='./datasets/ratings_for_upload.csv'):
rating_path='./datasets/ratings_for_upload.csv'
fin_data_path='2013q2'
print(fin_data_path, rating_path)

rating_feats = []
print('Loading rating data.')
ratings = pd.read_csv(rating_path)

rating_feats.append('rating_agency_name')
rating_feats.append('issuer_name')
rating_feats.append('rating_type')
rating_feats.append('rating')
rating_feats.append('rating_action_date')
rating_feats.append('file_creating_date')
rating_feats.append('maturity_date')
rating_feats.append('sec_category')
rating_feats.append('issuer_clean')

print('Loading financial num data.')
with open(f'./datasets/{fin_data_path}/num.txt') as op:
    df_num = pd.read_csv(StringIO(op.read()),sep='\t')

print('Loading financial sub data.')
with open(f'./datasets/{fin_data_path}/sub.txt') as op:
    df_sub = pd.read_csv(StringIO(op.read()),sep='\t')

adsh_to_name = dict(zip(df_sub.adsh, df_sub.name))

df_num['issuer_clean'] = df_num['adsh'].apply(adsh_to_name.get).apply(clear_issuer_name)

del df_sub

ratings['issuer_clean'] = ratings['issuer_name'].astype(str).apply(str.upper).apply(clear_issuer_name) 
_ratings = ratings[ratings['issuer_clean'].isin(set(df_num['issuer_clean']))].copy()
del ratings
ratings_per_company = {comp:[] for comp in _ratings['issuer_clean']}

for row in _ratings.itertuples(index=False):
    ratings_per_company[getattr(row, 'issuer_clean')].append(get_feats(row, rating_feats))

_sec_num = df_num[df_num['issuer_clean'].isin(set(_ratings['issuer_clean']))].copy()

del _ratings
del df_num

fin_statements = {k: defaultdict(dict) for k in set(_sec_num['issuer_clean'])}

print('Loading fin statements.')

for row in _sec_num.itertuples(index=False):
    tag = getattr(row, 'tag')
    qtrs = getattr(row, 'qtrs')
    dt = datetime.datetime.strptime(str(getattr(row, 'ddate')), '%Y%m%d')
    clear_issuer = clear_issuer_name(adsh_to_name[getattr(row, 'adsh')])

    fin_statements[clear_issuer][f'{dt.strftime("%Y_%m")}_{qtrs}Q'][tag] = getattr(row, 'value')

del _sec_num

fin_statements_per_year = defaultdict(dict)

for k in fin_statements:
    for year, group in groupby(sorted(fin_statements[k]), lambda s: s[:4]):
        fin_statements_per_year[k][year] = {}

        for statement in group:
            fin_statements_per_year[k][year].update(fin_statements[k][statement])

dataset = []

print('Creating dataset rows.')

for cmp, ratings in ratings_per_company.items():

    for r in ratings:
        r_year = r['rating_action_date'][0:4]

        if r_year not in fin_statements_per_year[cmp]:
            continue

        row = {}
        row['company'] = cmp
        row['rating'] = r['rating']
        row['rating_type'] = r['rating_type']
        row['rating_agency'] = r['rating_agency_name'] 
        row['year'] = r_year

        row.update(fin_statements_per_year[cmp][r_year])

        dataset.append(row)

del ratings_per_company
del fin_statements
del fin_statements_per_year

# return dataset


2013q2 ./datasets/ratings_for_upload.csv
Loading rating data.


/home/f/venvs/tcc/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (4,9,15,16,17,18,19,20,21,22,23,24,26,27,29,30,31) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


Loading financial num data.
Loading fin statements.
Creating dataset rows.


In [9]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [ ]:
dataset = []

for year, quarter in ((i,j) for i in range(2013,2016) for j in range(1,5)):
    print(len(dataset))
    print(f'{sys.getsizeof(dataset)*0.000001} Mbs\n')
    dataset.extend(load_dataset(load_dataset(f'{year}q{quarter}')))
    print(len(dataset))
    print(f'{sys.getsizeof(dataset)*0.000001} Mbs\n')

/tmp/ipykernel_17039/1598184525.py:4: DtypeWarning: Columns (4,9,15,16,17,18,19,20,21,22,23,24,26,27,29,30,31) have mixed types.Specify dtype option on import or set low_memory=False.
  dataset.extend(load_dataset(load_dataset(f'{year}q{quarter}')))


Loading data.
Loading fin statements.
Creating dataset rows.
Loading data.


In [3]:
[(i,j) for i in range(2013,2016) for j in range(1,5)]

[(2013, 1),
 (2013, 2),
 (2013, 3),
 (2013, 4),
 (2014, 1),
 (2014, 2),
 (2014, 3),
 (2014, 4),
 (2015, 1),
 (2015, 2),
 (2015, 3),
 (2015, 4)]

#### Reduce memory usage and work only with the necessary ammount of data.

In [61]:
sorted([y['rating_action_date'] for k in ratings_per_company for y in ratings_per_company[k] if y['rating_action_date'] > '2010-01-01'])

['2010-01-08',
 '2010-01-08',
 '2010-01-08',
 '2010-01-08',
 '2010-01-08',
 '2010-01-14',
 '2010-01-14',
 '2010-01-19',
 '2010-01-21',
 '2010-01-21',
 '2010-01-21',
 '2010-01-21',
 '2010-01-26',
 '2010-01-29',
 '2010-03-03',
 '2010-03-10',
 '2010-03-10',
 '2010-03-10',
 '2010-03-10',
 '2010-03-10',
 '2010-03-10',
 '2010-03-10',
 '2010-03-10',
 '2010-03-10',
 '2010-03-10',
 '2010-03-15',
 '2010-03-15',
 '2010-03-15',
 '2010-03-18',
 '2010-03-31',
 '2010-04-08',
 '2010-04-08',
 '2010-05-04',
 '2010-05-04',
 '2010-05-04',
 '2010-05-04',
 '2010-05-04',
 '2010-05-04',
 '2010-05-05',
 '2010-05-06',
 '2010-05-06',
 '2010-05-17',
 '2010-05-17',
 '2010-05-17',
 '2010-05-20',
 '2010-05-25',
 '2010-05-25',
 '2010-05-25',
 '2010-05-25',
 '2010-05-25',
 '2010-05-25',
 '2010-05-25',
 '2010-05-25',
 '2010-05-25',
 '2010-05-25',
 '2010-05-25',
 '2010-05-25',
 '2010-05-25',
 '2010-05-25',
 '2010-05-25',
 '2010-05-25',
 '2010-05-25',
 '2010-05-25',
 '2010-05-25',
 '2010-05-25',
 '2010-05-25',
 '2010-05-

In [50]:
tbt = ["Standard & Poor's Ratings Services", "Moody's Investors Service", "Fitch Ratings"]
ratings_per_company['APPLE']

[{'rating_agency_name': "Moody's Investors Service",
  'issuer_name': 'Apple Inc.',
  'rating_type': 'Instrument',
  'rating': 'Aa1',
  'rating_action_date': '2014-11-04',
  'file_creating_date': '2017-11-15',
  'maturity_date': '2022-11-10',
  'sec_category': 'Corporate',
  'issuer_clean': 'APPLE'},
 {'rating_agency_name': "Moody's Investors Service",
  'issuer_name': 'Apple Inc.',
  'rating_type': 'Instrument',
  'rating': 'Aa1',
  'rating_action_date': '2014-11-04',
  'file_creating_date': '2017-11-15',
  'maturity_date': '2026-11-10',
  'sec_category': 'Corporate',
  'issuer_clean': 'APPLE'},
 {'rating_agency_name': "Moody's Investors Service",
  'issuer_name': 'Apple Inc.',
  'rating_type': 'Instrument',
  'rating': 'Aa1',
  'rating_action_date': '2015-02-10',
  'file_creating_date': '2017-11-15',
  'maturity_date': '2030-02-25',
  'sec_category': 'Corporate',
  'issuer_clean': 'APPLE'},
 {'rating_agency_name': "Moody's Investors Service",
  'issuer_name': 'Apple Inc.',
  'rating_

In [75]:
pd.set_option("display.max_columns", None)
_ratings.loc[_ratings['issuer_clean'] == 'APPLE']

,rating_agency_name,file_creating_date,sec_category,issuer_name,legal_entity_identifier,object_type_rated,instrument_name,CUSIP_number,coupon_date,maturity_date,par_value,issued_paid,rating,rating_action_date,rating_action_class,rating_type,rating_sub_type,rating_type_term,other_announcement_type,watch_status,rating_outlook,issuer_identifier,issuer_identifier_schema,instrument_identifier,instrument_identifier_schema,central_index_key,obligor_identifier,obligor_identifier_schema,obligor_identifier_other,obligor_sec_category,obligor_industry_group,obligor_name,issuer_clean
4183588,Moody's Investors Service,2017-11-15,Corporate,Apple Inc.,HWUPKR0MPOU8FGXBT394,Instrument,EURO MEDIUM TERM NOTES,NaN,1.000,2022-11-10,1400.0,True,Aa1,2014-11-04,NW,Instrument,NaN,Senior Unsecured,NaN,NaN,NaN,NaN,NaN,824218577,NRSRO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,APPLE
4183589,Moody's Investors Service,2017-11-15,Corporate,Apple Inc.,HWUPKR0MPOU8FGXBT394,Instrument,GLOBAL NOTES,NaN,1.625,2026-11-10,1400.0,True,Aa1,2014-11-04,NW,Instrument,NaN,Senior Unsecured,NaN,NaN,NaN,NaN,NaN,824219955,NRSRO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,APPLE
4183590,Moody's Investors Service,2017-11-15,Corporate,Apple Inc.,HWUPKR0MPOU8FGXBT394,Instrument,SWISS NOTES,NaN,0.750,2030-02-25,375.0,True,Aa1,2015-02-10,NW,Instrument,NaN,Senior Unsecured,NaN,NaN,NaN,NaN,NaN,824355630,NRSRO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,APPLE
4183591,Moody's Investors Service,2017-11-15,Corporate,Apple Inc.,HWUPKR0MPOU8FGXBT394,Instrument,GLOBAL NOTES,NaN,0.350,2020-06-10,250000.0,True,Aa1,2015-06-01,NW,Instrument,NaN,Senior Unsecured,NaN,NaN,NaN,NaN,NaN,824540843,NRSRO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,APPLE
4183592,Moody's Investors Service,2017-11-15,Corporate,Apple Inc.,HWUPKR0MPOU8FGXBT394,Instrument,EUROBONDS,NaN,3.600,2042-07-31,500.0,True,Aa1,2015-07-24,NW,Instrument,NaN,Senior Unsecured,NaN,NaN,NaN,NaN,NaN,824629943,NRSRO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,APPLE
4183593,Moody's Investors Service,2017-11-15,Corporate,Apple Inc.,HWUPKR0MPOU8FGXBT394,Instrument,EUROBONDS,NaN,3.050,2029-07-31,750.0,True,Aa1,2015-07-24,NW,Instrument,NaN,Senior Unsecured,NaN,NaN,NaN,NaN,NaN,824629947,NRSRO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,APPLE
4183594,Moody's Investors Service,2017-11-15,Corporate,Apple Inc.,HWUPKR0MPOU8FGXBT394,Instrument,AUSTRALIAN NOTES,NaN,2.850,2019-08-28,400.0,True,Aa1,2015-08-20,NW,Instrument,NaN,Senior Unsecured,NaN,NaN,NaN,NaN,NaN,824668456,NRSRO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,APPLE
4183595,Moody's Investors Service,2017-11-15,Corporate,Apple Inc.,HWUPKR0MPOU8FGXBT394,Instrument,EURONOTES,NaN,1.375,2024-01-17,1000.0,True,Aa1,2015-09-10,NW,Instrument,NaN,Senior Unsecured,NaN,NaN,NaN,NaN,NaN,824701210,NRSRO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,APPLE
4183596,Moody's Investors Service,2017-11-15,Corporate,Apple Inc.,HWUPKR0MPOU8FGXBT394,Instrument,EURONOTES,NaN,2.000,2027-09-17,1000.0,True,Aa1,2015-09-10,NW,Instrument,NaN,Senior Unsecured,NaN,NaN,NaN,NaN,NaN,824707680,NRSRO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,APPLE
4183597,Moody's Investors Service,2017-11-15,Corporate,Apple Inc.,HWUPKR0MPOU8FGXBT394,Instrument,AUSTRALIAN NOTES,NaN,3.700,2022-08-28,1150.0,True,Aa1,2015-08-20,NW,Instrument,NaN,Senior Unsecured,NaN,NaN,NaN,NaN,NaN,824734563,NRSRO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,APPLE


In [76]:
_sec_num.loc[_sec_num['issuer_clean'] == 'APPLE']

,adsh,tag,version,coreg,ddate,qtrs,uom,value,footnote,issuer_clean
1465,0000320193-21-000065,NetIncomeLoss,us-gaap/2020,NaN,20210630,3,USD,7.412900e+10,NaN,APPLE
1466,0000320193-21-000065,NetIncomeLoss,us-gaap/2020,NaN,20210630,1,USD,2.174400e+10,NaN,APPLE
1467,0000320193-21-000065,NetIncomeLoss,us-gaap/2020,NaN,20200630,3,USD,4.473800e+10,NaN,APPLE
1468,0000320193-21-000065,NetIncomeLoss,us-gaap/2020,NaN,20200630,1,USD,1.125300e+10,NaN,APPLE
16337,0000320193-21-000065,PropertyPlantAndEquipmentNet,us-gaap/2020,NaN,20200930,0,USD,3.676600e+10,NaN,APPLE
...,...,...,...,...,...,...,...,...,...,...
2259110,0000320193-21-000065,OtherComprehensiveIncomeLossDerivativeInstrume...,0000320193-21-000065,NaN,20210630,1,USD,-1.700000e+07,NaN,APPLE
2259111,0000320193-21-000065,OtherComprehensiveIncomeLossDerivativeInstrume...,0000320193-21-000065,NaN,20200630,1,USD,1.120000e+09,NaN,APPLE
2259112,0000320193-21-000065,OtherComprehensiveIncomeLossDerivativeInstrume...,0000320193-21-000065,NaN,20210630,3,USD,-5.930000e+08,NaN,APPLE
2259113,0000320193-21-000065,OtherComprehensiveIncomeLossDerivativeInstrume...,0000320193-21-000065,NaN,20200630,3,USD,8.840000e+08,NaN,APPLE


In [67]:
with open('./datasets/2021q3/sub.txt') as op:
    df_sub = pd.read_csv(StringIO(op.read()),sep='\t')

In [78]:
df_sub.loc[df_sub['adsh'] == '0000320193-21-000065']

,adsh,cik,name,sic,countryba,stprba,cityba,zipba,bas1,bas2,baph,countryma,stprma,cityma,zipma,mas1,mas2,countryinc,stprinc,ein,former,changed,afs,wksi,fye,form,period,fy,fp,filed,accepted,prevrpt,detail,instance,nciks,aciks
3367,0000320193-21-000065,320193,APPLE INC,3571.0,US,CA,CUPERTINO,95014,ONE APPLE PARK WAY,NaN,(408) 996-1010,US,CA,CUPERTINO,95014,ONE APPLE PARK WAY,NaN,US,CA,942404110.0,APPLE INC,20070109.0,1-LAF,0,930.0,10-Q,20210630.0,2021.0,Q3,20210728,2021-07-27 18:04:00.0,0,1,aapl-20210626_htm.xml,1,NaN


In [91]:
import math
from collections import Counter
Counter([str(y['rating_action_date'])[:4] for k in ratings_per_company for y in ratings_per_company[k]])

Counter({'2013': 22125,
         '2015': 35135,
         '2014': 37339,
         '2016': 6756,
         '2012': 12474,
         '2010': 821,
         '2007': 12,
         '2008': 92,
         '2011': 2857,
         '1999': 7,
         '2005': 17,
         '2006': 42,
         '2009': 139,
         '1997': 6,
         '2004': 14,
         '2002': 2,
         '1993': 2,
         '1995': 5,
         '1996': 1,
         '1998': 2,
         '1994': 1,
         '2001': 4,
         '2003': 4,
         '2000': 2,
         '2017': 2})

In [14]:
fin_statements['APPLE']['2021_06_3Q']

{'NetIncomeLoss': 74129000000.0,
 'PaymentsOfDividends': 10827000000.0,
 'ProceedsFromPaymentsForOtherFinancingActivities': -72000000.0,
 'OtherComprehensiveIncomeLossDerivativeExcludedComponentIncreaseDecreaseBeforeAdjustmentsAndTax': -34000000.0,
 'WeightedAverageNumberDilutedSharesOutstandingAdjustment': 168871000.0,
 'IncreaseDecreaseInOtherOperatingAssets': 5899000000.0,
 'ProceedsFromRepaymentsOfShortTermDebtMaturingInThreeMonthsOrLess': 2745000000.0,
 'GrossProfit': 117662000000.0,
 'RepaymentsOfShortTermDebtMaturingInMoreThanThreeMonths': 3716000000.0,
 'IncreaseDecreaseInOtherReceivables': -4892000000.0,
 'InterestExpense': 1973000000.0,
 'NonoperatingIncomeExpense': 796000000.0,
 'WeightedAverageNumberOfSharesOutstandingBasic': 16772656000.0,
 'ProceedsFromSaleOfAvailableForSaleSecuritiesDebt': 36745000000.0,
 'NetCashProvidedByUsedInOperatingActivities': 83838000000.0,
 'OtherComprehensiveIncomeLossForeignCurrencyTransactionAndTranslationAdjustmentNetOfTax': 659000000.0,
 'P

In [17]:
fin_statements['BOEING']

defaultdict(dict,
            {'2021_06_0Q': {'AccountsPayableCurrent': 11450000000.0,
              'AccountsReceivableNetCurrent': 2490000000.0,
              'AccrualForEnvironmentalLossContingencies': 582000000.0,
              'AccruedLiabilitiesCurrent': 19502000000.0,
              'AccumulatedDepreciationDepletionAndAmortizationPropertyPlantAndEquipment': 20567000000.0,
              'AccumulatedOtherComprehensiveIncomeLossNetOfTax': -16661000000.0,
              'AdditionalPaidInCapital': 8481000000.0,
              'AllowanceForDoubtfulAccountsReceivable': 413000000.0,
              'Assets': 148935000000.0,
              'AssetsCurrent': 119095000000.0,
              'AvailableForSaleSecurities': 500000000.0,
              'CapitalizedComputerSoftwareAccumulatedAmortization': 863000000.0,
              'CashAndCashEquivalentsAtCarryingValue': 8271000000.0,
              'CashCashEquivalentsAndShortTermInvestments': 21300000000.0,
              'CashCashEquivalentsRestrictedC

In [94]:
import sys
sys.getsizeof(_ratings)

173366484

In [112]:
dataset[101]

{'company': 'ELECTRONIC ARTS',
 'rating': 'BBB-',
 'rating_type': 'Local Currency LT',
 'rating_agency': "Standard & Poor's Ratings Services",
 'year': '2016',
 'ProceedsFromIssuanceOfSeniorLongTermDebt': 395000000.0}

In [110]:
fin_statements_per_year['ELECTRONIC ARTS']

{'2016': {'ProceedsFromIssuanceOfSeniorLongTermDebt': 395000000.0},
 '2020': {'AccumulatedOtherComprehensiveIncomeLossNetOfTax': -52000000.0,
  'CashAndCashEquivalentsAtCarryingValue': 4013000000.0,
  'StockholdersEquity': 7782000000.0,
  'AdjustmentsToAdditionalPaidInCapitalSharebasedCompensationRequisiteServicePeriodRecognitionValue': 102000000.0,
  'AllocatedShareBasedCompensationExpense': 102000000.0,
  'AmortizationOfFinancingCosts': 1000000.0,
  'AmortizationOfIntangibleAssets': 5000000.0,
  'CashAndCashEquivalentsPeriodIncreaseDecrease': 245000000.0,
  'ComprehensiveIncomeNetOfTax': 363000000.0,
  'CostOfRevenue': 288000000.0,
  'DeferredRevenueRevenueRecognized1': 742000000.0,
  'Depreciation': 31000000.0,
  'DepreciationAmortizationAndAccretionNet': 37000000.0,
  'EarningsPerShareBasic': 1.27,
  'EarningsPerShareDiluted': 1.25,
  'EffectiveIncomeTaxRateContinuingOperations': 0.22,
  'EffectOfExchangeRateOnCashAndCashEquivalentsContinuingOperations': 19000000.0,
  'GeneralAndAd